In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import time
import pywt
import tensorflow as tf


from tensorflow import keras
from tensorflow.python.ops.nn_ops import conv1d
from keras.callbacks import Callback
from keras import backend as K
from keras.models import model_from_json
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding1D,Convolution1D
from tensorflow.keras.optimizers import SGD

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import FastICA
from pylab import *
from skimage import data, io, color

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
newpath= "/content/drive/MyDrive/dataset3"
A = "/content/drive/MyDrive/dataset3/A"
B = "/content/drive/MyDrive/dataset3/B"
C = "/content/drive/MyDrive/dataset3/C"
D = "/content/drive/MyDrive/dataset3/D"
E = "/content/drive/MyDrive/dataset3/E"
# F = "/content/drive/MyDrive/dataset3/F"
# G = "/content/drive/MyDrive/dataset3/G"
# I = "/content/drive/MyDrive/dataset3/H"
# J = "/content/drive/MyDrive/dataset3/I"
# K = "/content/drive/MyDrive/dataset3/J"
# L = "/content/drive/MyDrive/dataset3/K"
# M = "/content/drive/MyDrive/dataset3/L"
# N = "/content/drive/MyDrive/dataset3/M"
# N = "/content/drive/MyDrive/dataset3/N"
# O = "/content/drive/MyDrive/dataset3/O"
# P = "/content/drive/MyDrive/dataset3/P"
# Q = "/content/drive/MyDrive/dataset3/Q"
# R = "/content/drive/MyDrive/dataset3/R"
# S = "/content/drive/MyDrive/dataset3/S"
# T = "/content/drive/MyDrive/dataset3/T"
# U = "/content/drive/MyDrive/dataset3/U"
# V = "/content/drive/MyDrive/dataset3/V"
# W = "/content/drive/MyDrive/dataset3/W"
# X = "/content/drive/MyDrive/dataset3/X"
# Y = "/content/drive/MyDrive/dataset3/Y"
# Z = "/content/drive/MyDrive/dataset3/Z"

Fkelas =["A","B","C","D","E"]
numclasses=5

In [ ]:
#membuat dataset dari data latih tiap kelas
eegdatalatih=[]
def data_training():
    for kelas in Fkelas:
        pathkelas = os.path.join(newpath, kelas)
        numkelas = Fkelas.index(kelas)
        for edf in os.listdir(pathkelas):
            edf_array = pd.read_csv(os.path.join(pathkelas, edf))
            eegdatalatih.append([edf_array, numkelas])
    print("Jumlah data : ", len(eegdatalatih))


data_training()

Jumlah data :  110


In [ ]:
AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4 = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
beta,mu=[],[]
datalatih=[]
N_AF3, N_F7, N_F3, N_FC5, N_T7, N_P7, N_O1, N_O2, N_P8, N_T8, N_FC6, N_F4, N_F8, N_AF4 = [],[],[],[],[],[],[],[],[],[],[],[],[],[]
newdata=[]
vektor_input=[]
vektor_input1=[]
vektor_input_nowav=[]

def waveletcnn():
    for i in range(len(eegdatalatih)):
        AF3 = eegdatalatih[i][0][eegdatalatih[i][0].columns[0:1]].values
        F7 = eegdatalatih[i][0][eegdatalatih[i][0].columns[1:2]].values
        F3 = eegdatalatih[i][0][eegdatalatih[i][0].columns[2:3]].values
        FC5 = eegdatalatih[i][0][eegdatalatih[i][0].columns[3:4]].values
        T7 = eegdatalatih[i][0][eegdatalatih[i][0].columns[4:5]].values
        P7 = eegdatalatih[i][0][eegdatalatih[i][0].columns[5:6]].values
        O1 = eegdatalatih[i][0][eegdatalatih[i][0].columns[6:7]].values
        O2 = eegdatalatih[i][0][eegdatalatih[i][0].columns[7:8]].values
        P8 = eegdatalatih[i][0][eegdatalatih[i][0].columns[8:9]].values
        T8 = eegdatalatih[i][0][eegdatalatih[i][0].columns[9:10]].values
        FC6 = eegdatalatih[i][0][eegdatalatih[i][0].columns[10:11]].values
        F4 = eegdatalatih[i][0][eegdatalatih[i][0].columns[11:12]].values
        F8 = eegdatalatih[i][0][eegdatalatih[i][0].columns[12:13]].values
        AF4 = eegdatalatih[i][0][eegdatalatih[i][0].columns[13:14]].values

        # print(len(AF3))

        AF3 = np.reshape(AF3, 512)
        F7 = np.reshape(F7, 512)
        F3 = np.reshape(F3, 512)
        FC5 = np.reshape(FC5, 512)
        T7 = np.reshape(T7, 512)
        P7 = np.reshape(P7, 512)
        O1 = np.reshape(O1, 512)
        O2 = np.reshape(O2, 512)
        P8 = np.reshape(P8, 512)
        T8 = np.reshape(T8, 512)
        FC6 = np.reshape(FC6, 512)
        F4 = np.reshape(F4, 512)
        F8 = np.reshape(F8, 512)
        AF4 = np.reshape(AF4, 512)



        AF3 = minmax_scale(AF3, feature_range=(-1,1))
        F7 = minmax_scale(F7, feature_range=(-1,1))
        F3 = minmax_scale(F3, feature_range=(-1,1))
        FC5 = minmax_scale(FC5, feature_range=(-1,1))
        T7 = minmax_scale(T7, feature_range=(-1,1))
        P7 = minmax_scale(P7, feature_range=(-1,1))
        O1 = minmax_scale(O1, feature_range=(-1,1))
        O2 = minmax_scale(O2, feature_range=(-1,1))
        P8 = minmax_scale(P8, feature_range=(-1,1))
        T8 = minmax_scale(T8, feature_range=(-1,1))
        FC6 = minmax_scale(FC6, feature_range=(-1,1))
        F4 = minmax_scale(F4, feature_range=(-1,1))
        F8 =minmax_scale(F8, feature_range=(-1,1))
        AF4 = minmax_scale(AF4, feature_range=(-1,1))

        newdata= [AF3]+[F7]+[F3]+[FC5]+[T7]+[P7]+[O1]+[O2]+[P8]+[T8]+[FC6]+[F4]+[F8]+[AF4]
        gelombang=[]
        gelombang0=[]
        vektor_input1.append([np.concatenate((AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8, AF4),axis=None),eegdatalatih[i][1]])
        # print (newdata)
        # newdata=np.concatenate((af3,fc5,t7,t8,fc6,af4),axis=1)
        for j in range(14):
            wp = pywt.WaveletPacket(data = newdata[j], wavelet='db4', mode='per')
            # print(wp.data.shape)
            rentang1 = wp['aaaddd'].data #8Hz
            rentang2 = wp['aada'].data #9-12Hz
            rentang3 = wp['aaddaa'].data #13Hz
            rentang4 = wp['ada'].data #17-24Hz
            rentang5 = wp['adda'].data #25-28Hz
            rentang6 = wp['aaddd'].data #15-16Hz
            rentang7 = wp['addda'].data #29-30Hz
            rentang8 = wp['aadddd'].data #14Hz
            gelombang.append([np.concatenate((rentang1, rentang2, rentang3, rentang4, rentang5, rentang6, rentang7, rentang8),axis=None)])

        vektor_gelombang=np.concatenate((gelombang[0],gelombang[1],gelombang[2],gelombang[3],
                                         gelombang[4],gelombang[5],gelombang[6],gelombang[7],
                                         gelombang[8],gelombang[9],gelombang[10],gelombang[11],
                                         gelombang[12],gelombang[13]),axis=None)

        vektor_input.append([vektor_gelombang,eegdatalatih[i][1]])
waveletcnn()

In [ ]:
vektor_input[0][0].shape

(2576,)

In [ ]:
print("Jumlah data                              : ",len(vektor_input))             # banyaknya dataset
print("Panjang satu set data (Wavelet)          : ",len(vektor_input[0][0]))       # banyaknya data kalau pke wavelet (data 14 kanal)
print("Jumlah data                              : ",len(vektor_input1))            # banyaknya dataset
print("Panjang satu set data (Non Wavelet)      : ",len(vektor_input1[0][0]))      # banyaknya data kalau ga pke wavelet (data 14 kanal)

Jumlah data                              :  110
Panjang satu set data (Wavelet)          :  2576
Jumlah data                              :  110
Panjang satu set data (Non Wavelet)      :  7168


In [ ]:
## CNN Wavelet
conv1d_weights1= np.array([[[0.]],
                        [[1.]],
                        [[0.]]]).reshape((3,1,1))
for i  in range (len(vektor_input)):
  vektor_input[i][0] = np.array(vektor_input[i][0]).reshape(4, 14, 46, 1)
  data = tf.constant(vektor_input[i][0], dtype= tf.float32)
  # kernel = tf.constant(conv1d_weights1, dtype=tf.float32)
  res = Conv2D(filters=1, kernel_size=(3,3), activation='relu', input_shape=(4,14,16,1))(data)
  max_pool_2d = tf.keras.layers.MaxPooling2D(pool_size=(2,2), padding='valid')
  res2 = np.array(max_pool_2d(res))
  # data2 = tf.constant(res2, dtype= tf.float32)
  # res3 = Conv2D(1, (3,3), activation='relu', input_shape=(4,3,11,1))(data2)
  # max_pool_2d_2 = tf.keras.layers.MaxPooling2D(pool_size=(4,2), padding='valid')
  # res4 = np.array(max_pool_2d_2(res3))
  # # vektor_input[i][0] = res3
  vektor_input[i][0] = res2

In [ ]:
print(vektor_input[0][0].shape)

(4, 6, 22, 1)


In [ ]:
print("Persiapan data")
#Split features and label from DATASET
X,X1 = [],[] #features
Y,Y1 = [],[] #labels

for features,label in vektor_input:
    X.append(features)
    Y.append(label)

x_train,x_test,y_train,y_test= train_test_split(X, Y, train_size=0.8)

x_train = np.array(x_train)
# print(x_train.shape)
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],6,x_train.shape[3],1)
y_train = np.array(y_train)
x_test = np.array(x_test)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],6,x_test.shape[3],1)
y_test = np.array(y_test)


print("WAVELET")
print("Ukuran x_train :",x_train.shape)
print("Ukuran x_test  :",x_train.shape[1])
print("Ukuran x_test  :",x_test.shape)

Persiapan data
WAVELET
Ukuran x_train : (88, 4, 6, 22, 1)
Ukuran x_test  : 4
Ukuran x_test  : (22, 4, 6, 22, 1)


In [ ]:
#convert class vector to binary class metrices *once exec
y_train = keras.utils.to_categorical(y_train,numclasses)
y_test = keras.utils.to_categorical(y_test,numclasses)

print(y_train[0])

[1. 0. 0. 0. 0.]


In [ ]:
x_train.shape[1:]

(4, 6, 22, 1)

In [ ]:
x_train[4]

array([[[[2.76057810e-01],
         [4.17403340e-01],
         [3.06211054e-01],
         [1.56240538e-01],
         [2.42986083e-01],
         [1.08176693e-01],
         [6.53921664e-02],
         [1.82341605e-01],
         [4.19409394e-01],
         [1.80799395e-01],
         [3.04487288e-01],
         [3.12459558e-01],
         [5.07575631e-01],
         [2.13211164e-01],
         [1.58853754e-01],
         [1.66720033e-01],
         [2.03770176e-02],
         [2.45360926e-01],
         [2.76151866e-01],
         [4.42447484e-01],
         [1.57243133e-01],
         [6.10739961e-02]],

        [[1.25866011e-01],
         [4.57683802e-01],
         [7.06881046e-01],
         [4.35882270e-01],
         [0.00000000e+00],
         [5.81477165e-01],
         [3.11875612e-01],
         [1.89689159e-01],
         [4.53242734e-02],
         [1.57356456e-01],
         [1.29764646e-01],
         [2.46647894e-01],
         [2.29992390e-01],
         [4.34309125e-01],
         [2.86695540e-01],

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM

# LEARNING LSTM WITH Adam
modelWavelet = Sequential()
modelWavelet.add(LSTM(512, activation='relu', input_shape =(x_train.shape[1:]),return_sequences=True))
modelWavelet.add(Dropout(0.2))
modelWavelet.add(LSTM(512,activation='sigmoid'))
modelWavelet.add(Dropout(0.2))
modelWavelet.add(Dense(8, activation='softmax'))

# sgd = keras.optimizers.SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
# rms = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0)
# adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# adadelta = tf.keras.optimizers.Adadelta(learning_rate=0.02, rho=0.95, epsilon=1e-07)
modelWavelet.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelWavelet.summary()

ValueError: ignored

In [ ]:
start_time = time.time()
historyWavelet1 = modelWavelet.fit(x_train,y_train, epochs=200, batch_size =16, validation_data=(x_test,y_test),shuffle = False)
print("\nTraining model selesai")
print("Lama waktu learning : ",((time.time() - start_time)/60))